In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import h5py
import plotly
#import pyyaml
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import os
import numpy as np
import timeit
import time
import tensorflow as tf
import tensorflow.keras
import pickle as pkl
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Attention
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
#from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
#tf.compat.v1.disable_eager_execution()
np.set_printoptions(suppress=True)

In [ ]:
print(tf.__version__)

In [ ]:
!nvidia-smi -L

In [ ]:
%run metadata_proc.py

In [ ]:
# Read metadata #

dloc = '/Stanford_Dataset/chunk2.csv'
wloc = '/Stanford_Dataset/chunk2.hdf5'
d1, b = mda_proc(dloc)
w, m, wlen, mlen = wave_proc(d1, wloc)

In [ ]:
p1 = mag_plot(m)

In [ ]:
p2 = snr_plot(d1)

In [ ]:
%run trace_viz.py

In [ ]:
# Time domain plots #

tp = tplot(w)

In [ ]:
%run hann_taper.py
%run sigproc.py

In [ ]:
ht = myHanningTaper(w)
fq = fconv(ht, b)

In [ ]:
# Frequency domain plots #

fp = fplot(fq)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [ ]:
%run layers_class.py

In [ ]:
# Train Test Split #

from sklearn.model_selection import train_test_split

x1 = fq[:,:,0]
x2 = fq[:,:,1]
x3 = fq[:,:,2]

train_e, test_e, ytrain_e, ytest_e = train_test_split(x1, m, shuffle=True, test_size=0.1)
train_n, test_n, ytrain_n, ytest_n = train_test_split(x2, m, shuffle=True, test_size=0.1)
train_z, test_z, ytrain_z, ytest_z = train_test_split(x3, m, shuffle=True, test_size=0.1)

h = len(d1) - 0.1 * len(d1)

train_e = np.reshape(train_e, (0.1*len(d1),3001,1))
test_e = np.reshape(test_e, (h,3001,1))
train_n = np.reshape(train_n, (0.1*len(d1),3001,1))
test_n = np.reshape(test_n, (h,3001,1))
train_z = np.reshape(train_z, (0.1*len(d1),3001,1))
test_z = np.reshape(test_z, (h,3001,1))

print(train_e.shape, ytrain_e.shape, test_e.shape, ytest_e.shape)
print(train_n.shape, ytrain_n.shape, test_n.shape, ytest_n.shape)
print(train_z.shape, ytrain_z.shape, test_z.shape, ytest_z.shape)

# Training channel E

In [ ]:
epochs_number = 100
bach_size = 10
drop_rate = 0.2

#strategy = tf.distribute.MirroredStrategy()

#with strategy.scope():

inp = Input(shape=(3001,1), dtype='float32', name='input_layer') 
#inp = tf.cast(inp, tf.int32)
# Embed frequency data #
#freq_embed = F2V(750,3001)(inp)
#freq_embed = tf.cast(freq_embed, tf.float32)
#print(K.get_value(e[:50]))
encoder = Bidirectional(LSTM(50, return_sequences=True, dropout=0.0, recurrent_dropout=0.0))(inp)
#print(tf.shape(e))
#e = tf.reshape(e,(1500,200))
e1, awts, attn_out = attention1()(encoder)
#awts = awts.numpy()
#print(type(a))
#attn_out = tf.reshape(attn_out,(1500,200))
#e = tf.expand_dims(attn_out,axis=2)
print(tf.shape(attn_out))
#e = Flatten()(attn_out)
#print(tf.shape(e))
#e = tf.reshape(e,(1,200))
#print(tf.shape(e))
#e = tf.expand_dims(attn_out,axis=1)
#e = Conv1D(64,10,padding='same')(e)
#e = Dropout(drop_rate)(e, training=True)
#e = MaxPooling1D(4, padding='same')(e)
#e = tf.reshape(attn_out,(200,1))
#e = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(e)
#print(tf.shape(attn_out))
#print(tf.shape(e))
# Downsampling with CNN
#f = Conv1D(64, 1, padding = 'same')(inp) 
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#e = Conv1D(32, 1, padding = 'same')(f)
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#a, attn_out = attention()(e)
#print(tf.shape(attn_out))
#e = tf.keras.layers.RepeatVector(188)(attn_out)
#e = tf.expand_dims(attn_out, axis=2)

# Bi-LSTM for learning temporal dependencies
#f = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(f)

#print(tf.shape(e))

# Attention layer
#b, a, attn_out = attention()(e)
#print(tf.shape(attn_out))
# Connect with a dense layer for outputting magnitude (local)
d = Dense(1)(attn_out)
o = Activation('linear', name='output_layer')(d)

model = Model(inputs=[inp], outputs=o)

model.compile(optimizer='Adam', loss='MSE')

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                                patience= 4,
                               min_lr=0.5e-10
                              )
model.summary()

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,to_file='freq_mag.png')

In [ ]:
start = time.perf_counter()
history = model.fit(train_e, ytrain_e, epochs=epochs_number, validation_split=0.1, batch_size=bach_size, callbacks = [lr_reducer])
elap = time.perf_counter() - start
print('%.3f' % elap)

# Get attention scores
enc = model.layers[1]
attention_layer = model.layers[2]
encout = enc(train_e[0:1,:])
e2, awts, _ = attention_layer(encout)
#print(tf.reduce_max(awts),tf.reduce_min(awts))
awts = tf.reshape(e2,(3001,1))
train1 = tf.reshape(train_e[0:1],(3001,1))
#print(tf.shape(awts))
#v = yz[:1]
#v = tf.reshape(v,(3001,1))
#av = tf.concat((awts,train1),axis=1)
#print(tf.shape(av))
#merged = tf.stack((awts,train1),axis=1)
#print(tf.shape(merged))
fig, ax = plt.subplots(figsize=(10,10))
#fig, ax = plt.subplots(ncols=2, gridspec_kw=dict(width_ratios=[5,0.5]))
title = 'Attention Heatmap'
plt.title(title,fontsize=15)
#plt.setp(ax.get_xticklabels(), visible=False)
ttl = ax.title
#ax.axis('off')
ttl.set_position([0.5,1.05])
sns.heatmap(awts[:,:])#, annot=True, cbar=True)#, ax=axs[1])
#fig.colorbar(axs[1].collections[0], cax=axs[1])
plt.show()

In [ ]:
# Plot attention scores of respective frequencies

import plotly.express as px
sr = 100
fny = sr / 2.0
v = fq[0,:,0]
m = np.linspace(0,fny,len(v))

e3 = tf.reshape(e2,(3001))
awt = tf.reshape(awts,(3001))
#print(tf.shape(awts))
#print(m1[:5])
#print(awts[:5])
#print(awts.dtype)
#print(m1.dtype)
#k = tf.concat([m1,awts],0)
#k = tf.reshape(k,(3001,2))
print(tf.shape(e3))
e3 = e3.numpy()
awt = awt.numpy() 
e3 = e3.reshape((1,3001))
e3 = np.round(e3, 2)
awt = awt.reshape((1,3001))
#k = np.concatenate([m,e2])
print(e3.shape,awt.shape)
#k = k.reshape(3001,2)
#print(k.shape)
# Convert to dataframe
#kp = pd.DataFrame(k, columns=['Frequency [Hz]', 'Attention Scores'])
#print(kp.head)

# Plot
#fig = make_subplots(rows=1,cols=1,vertical_spacing=0.02,x_title='Frequency [Hz]',y_title='Attention Scores')
#fig.add_trace(go.Histogram2d(
 #   x=m,
  #  y=,
   # colorscale='YlGnBu',
#))

#amp = train_loc_z[0,:,]
mag = ytrain_e[0]
#mag = np.repeat(mag,3001)
#mag = np.reshape(mag, (3001,1))
#print(amp.shape, mag.shape)

#amp = amp.tolist()
m = np.round(m, 2)
m1 = m.tolist()
m2 = list(map(str,m1))
mag1 = mag.tolist()
mag2 = list(map(str,mag1))
print(len(m2), len(mag2))
#fig.update_xaxes(range=range(0.0,0.005,0.001))
#fig = px.density_heatmap(e2, x='Frequency [Hz]', y='Attention Scores', text_auto=True)

fig = px.imshow(e3[:,0:10],
                labels=dict(x='Magnitude',y='Frequency',color='Attention Scores'),
                x=m1[:10],
                y=mag1,
                text_auto=True
               )
fig.update_xaxes(side='top')
fig.show()

In [ ]:
# Loss Plots #

%matplotlib inline
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(history.history['loss'])
ax.plot(history.history['val_loss'], '--')
ax.legend(['loss', 'val_loss'], loc='upper right') 
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('E Channel Loss')
plt.rcParams['figure.figsize'] = (5,5)
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.show()

In [ ]:
# Evaluate

eva = model.evaluate(test_e, ytest_e)
print('test loss: ',eva)

predic = model.predict(test_e)

# Training channel N

In [ ]:
epochs_number = 100
bach_size = 10
drop_rate = 0.2

#strategy = tf.distribute.MirroredStrategy()

#with strategy.scope():

inp = Input(shape=(3001,1), dtype='float32', name='input_layer') 
#inp = tf.cast(inp, tf.int32)
# Embed frequency data #
#freq_embed = F2V(750,3001)(inp)
#freq_embed = tf.cast(freq_embed, tf.float32)
#print(K.get_value(e[:50]))
encoder = Bidirectional(LSTM(50, return_sequences=True, dropout=0.0, recurrent_dropout=0.0))(inp)
#print(tf.shape(e))
#e = tf.reshape(e,(1500,200))
e1, awts, attn_out = attention1()(encoder)
#awts = awts.numpy()
#print(type(a))
#attn_out = tf.reshape(attn_out,(1500,200))
#e = tf.expand_dims(attn_out,axis=2)
print(tf.shape(attn_out))
#e = Flatten()(attn_out)
#print(tf.shape(e))
#e = tf.reshape(e,(1,200))
#print(tf.shape(e))
#e = tf.expand_dims(attn_out,axis=1)
#e = Conv1D(64,10,padding='same')(e)
#e = Dropout(drop_rate)(e, training=True)
#e = MaxPooling1D(4, padding='same')(e)
#e = tf.reshape(attn_out,(200,1))
#e = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(e)
#print(tf.shape(attn_out))
#print(tf.shape(e))
# Downsampling with CNN
#f = Conv1D(64, 1, padding = 'same')(inp) 
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#e = Conv1D(32, 1, padding = 'same')(f)
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#a, attn_out = attention()(e)
#print(tf.shape(attn_out))
#e = tf.keras.layers.RepeatVector(188)(attn_out)
#e = tf.expand_dims(attn_out, axis=2)

# Bi-LSTM for learning temporal dependencies
#f = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(f)

#print(tf.shape(e))

# Attention layer
#b, a, attn_out = attention()(e)
#print(tf.shape(attn_out))
# Connect with a dense layer for outputting magnitude (local)
d = Dense(1)(attn_out)
o = Activation('linear', name='output_layer')(d)

model = Model(inputs=[inp], outputs=o)

model.compile(optimizer='Adam', loss='MSE')

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                                patience= 4,
                               min_lr=0.5e-10
                              )
model.summary()

In [ ]:
start = time.perf_counter()
history1 = model.fit(train_n, ytrain_n, epochs=epochs_number, validation_split=0.1, batch_size=bach_size, callbacks = [lr_reducer])
elap = time.perf_counter() - start
print('%.3f' % elap)

# Get attention scores
enc = model.layers[1]
attention_layer = model.layers[2]
encout = enc(train_e[0:1,:])
e2, awts, _ = attention_layer(encout)
#print(tf.reduce_max(awts),tf.reduce_min(awts))
awts = tf.reshape(e2,(3001,1))
train1 = tf.reshape(train_e[0:1],(3001,1))
#print(tf.shape(awts))
#v = yz[:1]
#v = tf.reshape(v,(3001,1))
#av = tf.concat((awts,train1),axis=1)
#print(tf.shape(av))
#merged = tf.stack((awts,train1),axis=1)
#print(tf.shape(merged))
fig, ax = plt.subplots(figsize=(10,10))
#fig, ax = plt.subplots(ncols=2, gridspec_kw=dict(width_ratios=[5,0.5]))
title = 'Attention Heatmap'
plt.title(title,fontsize=15)
#plt.setp(ax.get_xticklabels(), visible=False)
ttl = ax.title
#ax.axis('off')
ttl.set_position([0.5,1.05])
sns.heatmap(awts[:,:])#, annot=True, cbar=True)#, ax=axs[1])
#fig.colorbar(axs[1].collections[0], cax=axs[1])
plt.show()

In [ ]:
# Plot attention scores of respective frequencies

import plotly.express as px
sr = 100
fny = sr / 2.0
v = fq[0,:,1]
m = np.linspace(0,fny,len(v))

e3 = tf.reshape(e2,(3001))
awt = tf.reshape(awts,(3001))
#print(tf.shape(awts))
#print(m1[:5])
#print(awts[:5])
#print(awts.dtype)
#print(m1.dtype)
#k = tf.concat([m1,awts],0)
#k = tf.reshape(k,(3001,2))
print(tf.shape(e3))
e3 = e3.numpy()
awt = awt.numpy() 
e3 = e3.reshape((1,3001))
e3 = np.round(e3,2)
awt = awt.reshape((1,3001))
#k = np.concatenate([m,e2])
print(e3.shape,awt.shape)
#k = k.reshape(3001,2)
#print(k.shape)
# Convert to dataframe
#kp = pd.DataFrame(k, columns=['Frequency [Hz]', 'Attention Scores'])
#print(kp.head)

# Plot
#fig = make_subplots(rows=1,cols=1,vertical_spacing=0.02,x_title='Frequency [Hz]',y_title='Attention Scores')
#fig.add_trace(go.Histogram2d(
 #   x=m,
  #  y=,
   # colorscale='YlGnBu',
#))

#amp = train_loc_z[0,:,]
mag = ytrain_n[0]
#mag = np.repeat(mag,3001)
#mag = np.reshape(mag, (3001,1))
#print(amp.shape, mag.shape)

#amp = amp.tolist()
m = np.round(m,2)
m1 = m.tolist()
m2 = list(map(str,m1))
mag1 = mag.tolist()
mag2 = list(map(str,mag1))
print(len(m2), len(mag2))
#fig.update_xaxes(range=range(0.0,0.005,0.001))
#fig = px.density_heatmap(e2, x='Frequency [Hz]', y='Attention Scores', text_auto=True)

fig = px.imshow(e3[:,0:10],
                labels=dict(x='Magnitude',y='Frequency',color='Attention Scores'),
                x=m1[:10],
                y=mag1,
                text_auto=True
               )
fig.update_xaxes(side='top')
fig.show()

In [ ]:
# Loss Plots #

%matplotlib inline
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(history1.history['loss'])
ax.plot(history1.history['val_loss'], '--')
ax.legend(['loss', 'val_loss'], loc='upper right') 
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('N Channel Loss')
plt.rcParams['figure.figsize'] = (5,5)
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.show()

In [ ]:
# Evaluate

eva1 = model.evaluate(test_n, ytest_n)
print('test loss: ',eva1)

predic1 = model.predict(test_n)

# Training channel Z

In [ ]:
epochs_number = 100
bach_size = 10
drop_rate = 0.2

#strategy = tf.distribute.MirroredStrategy()

#with strategy.scope():

inp = Input(shape=(3001,1), dtype='float32', name='input_layer') 
#inp = tf.cast(inp, tf.int32)
# Embed frequency data #
#freq_embed = F2V(750,3001)(inp)
#freq_embed = tf.cast(freq_embed, tf.float32)
#print(K.get_value(e[:50]))
encoder = Bidirectional(LSTM(50, return_sequences=True, dropout=0.0, recurrent_dropout=0.0))(inp)
#print(tf.shape(e))
#e = tf.reshape(e,(1500,200))
e1, awts, attn_out = attention1()(encoder)
#awts = awts.numpy()
#print(type(a))
#attn_out = tf.reshape(attn_out,(1500,200))
#e = tf.expand_dims(attn_out,axis=2)
print(tf.shape(attn_out))
#e = Flatten()(attn_out)
#print(tf.shape(e))
#e = tf.reshape(e,(1,200))
#print(tf.shape(e))
#e = tf.expand_dims(attn_out,axis=1)
#e = Conv1D(64,10,padding='same')(e)
#e = Dropout(drop_rate)(e, training=True)
#e = MaxPooling1D(4, padding='same')(e)
#e = tf.reshape(attn_out,(200,1))
#e = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(e)
#print(tf.shape(attn_out))
#print(tf.shape(e))
# Downsampling with CNN
#f = Conv1D(64, 1, padding = 'same')(inp) 
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#e = Conv1D(32, 1, padding = 'same')(f)
#print(tf.shape(e))
#f = Dropout(drop_rate)(f, training=True)
#f = MaxPooling1D(4, padding='same')(f)
#print(tf.shape(e))
#a, attn_out = attention()(e)
#print(tf.shape(attn_out))
#e = tf.keras.layers.RepeatVector(188)(attn_out)
#e = tf.expand_dims(attn_out, axis=2)

# Bi-LSTM for learning temporal dependencies
#f = Bidirectional(LSTM(100, return_sequences=False, dropout=0.0, recurrent_dropout=0.0))(f)

#print(tf.shape(e))

# Attention layer
#b, a, attn_out = attention()(e)
#print(tf.shape(attn_out))
# Connect with a dense layer for outputting magnitude (local)
d = Dense(1)(attn_out)
o = Activation('linear', name='output_layer')(d)

model = Model(inputs=[inp], outputs=o)

model.compile(optimizer='Adam', loss='MSE')

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                                patience= 4,
                               min_lr=0.5e-10
                              )
model.summary()

In [ ]:
start = time.perf_counter()
history2 = model.fit(train_z, ytrain_z, epochs=epochs_number, validation_split=0.1, batch_size=bach_size, callbacks = [lr_reducer])
elap = time.perf_counter() - start
print('%.3f' % elap)

# Get attention scores
enc = model.layers[1]
attention_layer = model.layers[2]
encout = enc(train_e[0:1,:])
e2, awts, _ = attention_layer(encout)
#print(tf.reduce_max(awts),tf.reduce_min(awts))
awts = tf.reshape(e2,(3001,1))
train1 = tf.reshape(train_e[0:1],(3001,1))
#print(tf.shape(awts))
#v = yz[:1]
#v = tf.reshape(v,(3001,1))
#av = tf.concat((awts,train1),axis=1)
#print(tf.shape(av))
#merged = tf.stack((awts,train1),axis=1)
#print(tf.shape(merged))
fig, ax = plt.subplots(figsize=(10,10))
#fig, ax = plt.subplots(ncols=2, gridspec_kw=dict(width_ratios=[5,0.5]))
title = 'Attention Heatmap'
plt.title(title,fontsize=15)
#plt.setp(ax.get_xticklabels(), visible=False)
ttl = ax.title
#ax.axis('off')
ttl.set_position([0.5,1.05])
sns.heatmap(awts[:,:])#, annot=True, cbar=True)#, ax=axs[1])
#fig.colorbar(axs[1].collections[0], cax=axs[1])
plt.show()

In [ ]:
# Plot attention scores of respective frequencies

import plotly.express as px
sr = 100
fny = sr / 2.0
v = fq[0,:,2]
m = np.linspace(0,fny,len(v))

e3 = tf.reshape(e2,(3001))
awt = tf.reshape(awts,(3001))
#print(tf.shape(awts))
#print(m1[:5])
#print(awts[:5])
#print(awts.dtype)
#print(m1.dtype)
#k = tf.concat([m1,awts],0)
#k = tf.reshape(k,(3001,2))
print(tf.shape(e3))
e3 = e3.numpy()
awt = awt.numpy() 
e3 = e3.reshape((1,3001))
e3 = np.round(e3,2)
awt = awt.reshape((1,3001))
#k = np.concatenate([m,e2])
print(e3.shape,awt.shape)
#k = k.reshape(3001,2)
#print(k.shape)
# Convert to dataframe
#kp = pd.DataFrame(k, columns=['Frequency [Hz]', 'Attention Scores'])
#print(kp.head)

# Plot
#fig = make_subplots(rows=1,cols=1,vertical_spacing=0.02,x_title='Frequency [Hz]',y_title='Attention Scores')
#fig.add_trace(go.Histogram2d(
 #   x=m,
  #  y=,
   # colorscale='YlGnBu',
#))

#amp = train_loc_z[0,:,]
mag = ytrain_z[0]
#mag = np.repeat(mag,3001)
#mag = np.reshape(mag, (3001,1))
#print(amp.shape, mag.shape)

#amp = amp.tolist()
m = np.round(m,2)
m1 = m.tolist()
m2 = list(map(str,m1))
mag1 = mag.tolist()
mag2 = list(map(str,mag1))
print(len(m2), len(mag2))
#fig.update_xaxes(range=range(0.0,0.005,0.001))
#fig = px.density_heatmap(e2, x='Frequency [Hz]', y='Attention Scores', text_auto=True)

fig = px.imshow(e3[:,0:10],
                labels=dict(x='Magnitude',y='Frequency',color='Attention Scores'),
                x=m1[:10],
                y=mag1,
                text_auto=True
               )
fig.update_xaxes(side='top')
fig.show()

In [ ]:
# Loss Plots #

%matplotlib inline
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(history2.history['loss'])
ax.plot(history2.history['val_loss'], '--')
ax.legend(['loss', 'val_loss'], loc='upper right') 
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Z Channel Loss')
plt.rcParams['figure.figsize'] = (5,5)
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.show()

In [ ]:
# Evaluate

eva2 = model.evaluate(test_z, ytest_z)
print('test loss: ',eva1)

predic2 = model.predict(test_z)